<a href="https://colab.research.google.com/github/IzzyBrand/robotBottleFlip/blob/main/release_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import ipywidgets
import pydot
import os
from IPython.display import display, SVG

import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='20201120', drake_build='nightly')

if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

# Use pyngrok on colab.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import pydrake.all
from manipulation.scenarios import AddIiwa, AddWsg, AddRgbdSensors, AddShape
from manipulation.utils import FindResource

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator, TrajectorySource)

from pydrake.all import (
    DiagramBuilder,
    RenderEngineVtkParams, MakeRenderEngineVtk, DepthCameraProperties, RgbdSensor, RigidTransform, RollPitchYaw,
    ColorRenderCamera, DepthRenderCamera, CameraInfo, ClippingRange, RenderCameraCore, DepthRange
)
    
from pydrake.all import RigidTransform, RotationMatrix, SpatialVelocity, PiecewisePolynomial

from pydrake.all import eq, MathematicalProgram, Solve

from pydrake.all import RenderEngineVtkParams, DepthCameraProperties, ModelInstanceIndex, DepthImageToPointCloud, BaseField, LeafSystem, AbstractValue

from google.colab.patches import cv2_imshow

In [ ]:
import requests
resp = requests.get("https://raw.githubusercontent.com/IzzyBrand/robotBottleFlip/main/models/bottle.sdf")
if resp.status_code == 200:
    bottle_sdf = resp.text

In [ ]:
T = 500 # number of timesteps
d = 12 # state dimension

def dynamics(x):
    dx = np.zeros_like(x)
    dx[0:3] = x[3:6] # velocity
    dx[5] = -9.81    # acceleration
    dx[6:9] = x[9:] # angular velocity
    return dx

prog = MathematicalProgram()
# create decision variables for the state and the timestep
state = prog.NewContinuousVariables(T + 1, d, 'state')
dt = prog.NewContinuousVariables(1, 'dt')

# bound the starting and ending positions
prog.AddLinearConstraint(eq(state[0,:3], np.array([1,0,0.5]))) # constrain initial position
prog.AddLinearConstraint(eq(state[-1,:3], np.array([3,0,0]))) # constrain final position
prog.AddLinearConstraint(eq(state[0, 6:9], np.zeros(3))) # constrain initial orientation
prog.AddLinearConstraint(eq(state[-1, 6:9], np.array([0,-2*np.pi-1e-3, 0]))) # constrain final orientation
# force it to be rotating backwards
# prog.AddBoundingBoxConstraint(np.array([0,-10,0]), np.array([0,-1,0]), state[0,9:]) # constrain intial rotational velocity

# Bound the timestep
prog.AddBoundingBoxConstraint(0.002, 0.05, dt[0])
# Do timestepping. Easy to do fixed timesteps with symplectic Euler
for t in range(T):
	prog.AddConstraint(eq(state[t+1], state[t] + dt[0]*dynamics(state[t])))


bottle_radius = 0.03
bottle_height_of_cg = 0.075
bottle_I_yy = 0.00105
bottle_mass = 0.5
sim_timestep = 0.002

angular_momentum = bottle_I_yy * state[0,10]
impact_torque = bottle_radius * bottle_mass * state[-1,5]
friction_torque = -bottle_height_of_cg * bottle_mass * state[-1, 3]
torque = impact_torque + friction_torque

prog.AddConstraint(angular_momentum == torque*sim_timestep)


result = Solve(prog)
assert(result.is_success()), "Optimization failed"
x0 = result.GetSolution(state)[0]
xT = result.GetSolution(state)[-1]
print(f'Release Velocity {x0[3:6]}')
print(f'Release Angular Velocity {x0[9:]}')
print(f'Timestep {result.GetSolution(dt)}')

In [ ]:
def create_world(target_platform_translation=[2.5,0,-.005], time_step=0.001):
    builder = pydrake.systems.framework.DiagramBuilder()
    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(
        builder, time_step=time_step)

    # add the bottle
    pydrake.multibody.parsing.Parser(plant).AddModelFromString(bottle_sdf, 'sdf')

    # add platforms for the bottle to sit
    AddShape(plant, pydrake.geometry.Box(0.2, 0.2, 0.01), 'target_platform')
    AddShape(plant, pydrake.geometry.Box(0.2, 0.2, 0.01), 'initial_platform')
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("target_platform"),
                        RigidTransform(RotationMatrix(), target_platform_translation))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("initial_platform"),
                        RigidTransform(RotationMatrix(), [0.5,0,-.005]))

    # put the bottle on the platform
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName('bottle_base_link'),
                            RigidTransform(RotationMatrix(), [0.5,0,0]))

    plant.Finalize()

    builder.ExportOutput(scene_graph.get_query_output_port(), "geometry_query")
    builder.ExportOutput(plant.get_contact_results_output_port(), "contact_results")
    builder.ExportOutput(plant.get_state_output_port(), "plant_continuous_state")

    return builder.Build()

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()
station_diagram = builder.AddSystem(create_world())
meshcat = ConnectMeshcatVisualizer(builder,
                                   output_port=station_diagram.GetOutputPort("geometry_query"),
                                   zmq_url=zmq_url)
diagram = builder.Build()
context = diagram.CreateDefaultContext()

simulator = Simulator(diagram)
plant = station_diagram.GetSubsystemByName('plant')
mutable_context = simulator.get_mutable_context()
mutable_plant_context = plant.GetMyMutableContextFromRoot(mutable_context)

plant.SetFreeBodyPose(mutable_context,
                      plant.GetBodyByName('bottle_base_link'),
                      RigidTransform(RotationMatrix(), [1,0,0.5]))
plant.SetFreeBodySpatialVelocity(plant.GetBodyByName('bottle_base_link'),
                                 SpatialVelocity(w=[ 0.,         -6.5135561,  0.        ],
                                                 v=[2.2277771,  0.,         3.25951315]),
                                 mutable_context)

# meshcat.load(meshcat.load(meshcat.GetMyContextFromRoot(context)))

In [ ]:
meshcat.start_recording()
simulator.AdvanceTo(3)
meshcat.publish_recording()